In [0]:
%run "/Users/ovidiumtoma@gmail.com/wind_turbine_project/src/wt_transformations"

In [0]:
transformer = DataTransformer(spark)

silver_table_name = "wind_turbine_silver"
df_silver = spark.read.table(f"silver_data.{silver_table_name}")

df_summary, df_anomalies = transformer.transform_and_save(df_silver)

# Show results
df_summary.show(15)
df_anomalies.show(15)


2025-02-21 01:23:30 - INFO - DataTransformer initialized.
2025-02-21 01:23:30 - INFO - Starting transformation process...
2025-02-21 01:23:30 - INFO - Summarizing power output...
2025-02-21 01:23:31 - INFO - Summary statistics calculated. Row count: 15
2025-02-21 01:23:31 - INFO - Identifying anomalies...
2025-02-21 01:23:34 - INFO - Anomaly detection complete. Anomalies found: 0
2025-02-21 01:23:34 - INFO - Saving transformed data to gold_data.turbine_summary...
2025-02-21 01:23:39 - INFO - Successfully saved transformed data to gold_data.turbine_summary
2025-02-21 01:23:39 - INFO - Saving transformed data to gold_data.turbine_anomalies...
2025-02-21 01:23:44 - INFO - Successfully saved transformed data to gold_data.turbine_anomalies
2025-02-21 01:23:44 - INFO - Transformation process complete.
+----------+---------+---------+------------------+------------------+
|turbine_id|min_power|max_power|         avg_power|      stddev_power|
+----------+---------+---------+------------------+